# Tensorflow X Keras: Buiding Blocks (How to Play)

This is a summary of the Keras basic building procedures, which is a guide of running a model with Tensorflow backend.

**The whole procedure can be divided into four parts:**
1. Create Model
2. Compile Backward Propagation Setting 
3. Train Model
4. Evaluate and Predict
5. Callback for TensorBoard (Visualization)

## 1. Create Model

For any model, there are two ways to create:
- [Functional API](https://keras.io/getting-started/functional-api-guide/) *--> define calculation between layers and assemble to a model*
    - highly customized computation graph
    - model is callable
    - model can be easily reused as a block for other model
- [Sequential Model API](https://keras.io/getting-started/sequential-model-guide/) *--> add layers upon model*
    - fast and easy to create
    - simple or standard computation graph
    
The shape of input need to be defined in 1st layer, and there is no sample size or batch size needed normally.  [Keras Doc](https://keras.io/getting-started/sequential-model-guide/#specifying-the-input-shape)

In [ ]:
### For Functional API                                          ### For Sequential Model API
from keras.models import Model                                  from keras.models import Sequential
from keras.layers import Input, Dense                           from keras.layers import Dense, Activation

                                                                # Start a model instance
                                                                model = Sequential()
# Define input shape by returning a tensor
inputs = Input(shape=(784,))

# Define layers and its inputs
# Use built-in functions or customized function returning x    # Top-up layers
x = Dense(32, activation='relu')(inputs)                        model.add(Dense(32, input_shape=(784,)))  # Only define on 1st
                                                                model.add(Activation('relu'))
x = Dense(32, activation='relu')(x)                             model.add(Dense(32))
                                                                model.add(Activation('relu'))
predictions = Dense(10, activation='softmax')(x)                model.add(Dense(10))
                                                                model.add(Activation('softmax'))
# Creates a model gathering above
model = Model(inputs=inputs, outputs=predictions)

## 2. Compile Backward Propagation Setting

This section includes three parts in normal case:
- **optimizer:** string or instance (created to customize setting)
- **loss:** string or objective function 
- **metrics:** a list of string or customized metrics function

If you are going to train a multi-input and multi-output model, the weights of different loss can be defined. [Keras Doc](https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models)

In [ ]:
### Default optimizer
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')


### Customized optimizer
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

## 3. Train Model

Define data, labels, epochs and batch_size for *model.fit*.

**PS:** For deep learning, there is no necessary to perform *cross-validation*. Because it is designed to search best hyperparameters for tiny dataset, and trains multiple models with different folds of dataset avoiding overfitting. In DL era, data is large enough to avoid overfitting so dataset is only splitted into **train / validate / test set**.

In [ ]:
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)

## 4. Evaluate and Predict

For special case, batch_size can be added in to keep the evaluate process same as training.

In [ ]:
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=128)
predict = model.predict(x_test,
                        batch_size=128)

## 5. Callback for TensorBoard (Visualization)

Create a callback instance as below and put into `model.fit(callbacks=<instance>)`.

``` python
keras.callbacks.TensorBoard(log_dir='./logs',
                            histogram_freq=0, 
                            batch_size=32, 
                            write_graph=True, 
                            write_grads=False, 
                            write_images=False, 
                            embeddings_freq=0, 
                            embeddings_layer_names=None, 
                            embeddings_metadata=None)
```

Run TensorBoard with `tensorboard --logdir=/full_path_to_your_logs`.

## Additional: Visualize the Model

In [ ]:
### List of model details
model.summary()


### Graphical demonstration using graphviz (export a file)
from keras.utils import plot_model

plot_model(model, to_file='model.png')


### Graphical demonstration using inline plotting
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

## More Example

### Residual connection on a convolution layer

In [1]:
from keras.layers import Conv2D, Input

# input tensor for a 3-channel 256x256 image
x = Input(shape=(256, 256, 3))
# 3x3 conv with 3 output channels (same as input channels)
y = Conv2D(3, (3, 3), padding='same')(x)
# this returns x + y.
z = keras.layers.add([x, y])

ImportError: No module named 'keras'

### Inception module

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

### Multilayer Perceptron (MLP) for multi-class softmax classification

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

### VGG-like convnet

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)